# Chapter 4: Decorators

Decorators are functions that modify or enhance functions/classes without permanently changing them. They provide a clean way to add functionality like logging, caching, authentication, and timing.

## Section 1: Understanding Decorators

In [ ]:
# Decorators are higher-order functions: functions that take functions as input
def my_decorator(func):
    """A simple decorator that prints when a function is called."""
    def wrapper():
        print("Before function call")
        func()
        print("After function call")
    return wrapper

# Manual decoration
def greet():
    print("Hello!")

decorated_greet = my_decorator(greet)
decorated_greet()

print("\n---\n")

# Using @ syntax (syntactic sugar)
@my_decorator
def say_goodbye():
    print("Goodbye!")

say_goodbye()

In [ ]:
# Decorators with arguments
def decorator_with_args(func):
    def wrapper(*args, **kwargs):
        print(f"Function '{func.__name__}' called with args={args}, kwargs={kwargs}")
        result = func(*args, **kwargs)
        print(f"Function returned: {result}")
        return result
    return wrapper

@decorator_with_args
def add(a, b):
    """Add two numbers."""
    return a + b

result = add(3, 5)
print()
result = add(a=10, b=20)

## Section 2: functools.wraps

In [ ]:
# Problem: decorator loses function metadata
def bad_decorator(func):
    def wrapper():
        return func()
    return wrapper

@bad_decorator
def important_function():
    """This function does something important."""
    return "result"

print(f"Name: {important_function.__name__}")
print(f"Docstring: {important_function.__doc__}")
# Output: 'wrapper' and None (metadata lost!)

In [ ]:
# Solution: use functools.wraps
from functools import wraps

def good_decorator(func):
    @wraps(func)  # Preserves metadata
    def wrapper(*args, **kwargs):
        print("Calling function...")
        return func(*args, **kwargs)
    return wrapper

@good_decorator
def important_function():
    """This function does something important."""
    return "result"

print(f"Name: {important_function.__name__}")
print(f"Docstring: {important_function.__doc__}")
# Output: 'important_function' and 'This function ...'

## Section 3: Parametrized Decorators

In [ ]:
# Decorators that take parameters
from functools import wraps

def repeat(times: int):
    """Repeat function execution."""
    def decorator(func):
        @wraps(func)
        def wrapper(*args, **kwargs):
            results = []
            for i in range(times):
                result = func(*args, **kwargs)
                results.append(result)
            return results
        return wrapper
    return decorator

@repeat(times=3)
def greet(name: str) -> str:
    return f"Hello, {name}!"

results = greet("Alice")
for i, result in enumerate(results, 1):
    print(f"{i}. {result}")

In [ ]:
# Rate limiting decorator
from functools import wraps
import time

def rate_limit(calls_per_second: float):
    """Limit how many times a function can be called."""
    min_interval = 1.0 / calls_per_second
    
    def decorator(func):
        last_called = [0.0]
        
        @wraps(func)
        def wrapper(*args, **kwargs):
            elapsed = time.time() - last_called[0]
            if elapsed < min_interval:
                time.sleep(min_interval - elapsed)
            
            result = func(*args, **kwargs)
            last_called[0] = time.time()
            return result
        return wrapper
    return decorator

@rate_limit(calls_per_second=2)
def api_call():
    print(f"API called at {time.time():.2f}")

# Calls are rate limited
for i in range(4):
    api_call()

## Section 4: Practical Decorators

# Caching/memoization decorator
from functools import wraps

def cache(func):
    """Cache function results."""
    cached_results = {}
    
    @wraps(func)
    def wrapper(*args):
        if args in cached_results:
            print(f"Cache hit for {args}")
            return cached_results[args]
        
        print(f"Cache miss for {args}, computing...")
        result = func(*args)
        cached_results[args] = result
        return result
    
    wrapper.cache_clear = lambda: cached_results.clear()
    wrapper.cache_info = lambda: cached_results.copy()
    return wrapper

@cache
def fibonacci(n: int) -> int:
    """Compute nth Fibonacci number."""
    if n < 2:
        return n
    return fibonacci(n-1) + fibonacci(n-2)

print(fibonacci(5))
print(fibonacci(5))  # Cache hit
print(f"\nCache info: {fibonacci.cache_info()}")

# Validation decorator
from functools import wraps

def validate_positive(*param_names):
    """Validate that specified parameters are positive."""
    def decorator(func):
        @wraps(func)
        def wrapper(*args, **kwargs):
            # Check keyword arguments
            for name in param_names:
                if name in kwargs:
                    if not isinstance(kwargs[name], (int, float)) or kwargs[name] <= 0:
                        raise ValueError(f"{name} must be positive")
            return func(*args, **kwargs)
        return wrapper
    return decorator

@validate_positive('x', 'y')
def divide(x, y=1):
    return x / y

print(divide(10, y=2))

# This will raise ValueError
try:
    divide(10, y=-2)
except ValueError as e:
    print(f"Error: {e}")

# Class decorator: add methods/attributes
def add_repr(cls):
    """Add a __repr__ method to a class."""
    def __repr__(self):
        attrs = ', '.join(f"{k}={v!r}" for k, v in self.__dict__.items())
        return f"{cls.__name__}({attrs})"
    
    cls.__repr__ = __repr__
    return cls

@add_repr
class Person:
    def __init__(self, name: str, age: int):
        self.name = name
        self.age = age

person = Person("Alice", 30)
print(person)
print(repr(person))

# Dataclass-like decorator
def dataclass(cls):
    """Simplified dataclass: generate __init__, __repr__, __eq__."""
    # Get type hints if available
    hints = getattr(cls, '__annotations__', {})
    
    def __init__(self, **kwargs):
        for key, value in kwargs.items():
            setattr(self, key, value)
    
    def __repr__(self):
        items = ', '.join(f"{k}={getattr(self, k, None)!r}" for k in hints)
        return f"{cls.__name__}({items})"
    
    def __eq__(self, other):
        if not isinstance(other, cls):
            return NotImplemented
        return all(getattr(self, k) == getattr(other, k) for k in hints)
    
    cls.__init__ = __init__
    cls.__repr__ = __repr__
    cls.__eq__ = __eq__
    return cls

@dataclass
class Point:
    x: int
    y: int

p1 = Point(x=1, y=2)
p2 = Point(x=1, y=2)
p3 = Point(x=3, y=4)

print(p1)
print(f"p1 == p2: {p1 == p2}")
print(f"p1 == p3: {p1 == p3}")

# Method decorator: validate method arguments
from functools import wraps

def require_type(**type_checks):
    """Validate argument types for a method."""
    def decorator(func):
        @wraps(func)
        def wrapper(*args, **kwargs):
            # Check kwargs types
            for name, expected_type in type_checks.items():
                if name in kwargs:
                    if not isinstance(kwargs[name], expected_type):
                        raise TypeError(f"{name} must be {expected_type.__name__}")
            return func(*args, **kwargs)
        return wrapper
    return decorator

class Calculator:
    @require_type(a=int, b=int)
    def add(self, a, b):
        return a + b

calc = Calculator()
print(calc.add(5, 3))

# Type check fails
try:
    calc.add(a="5", b=3)
except TypeError as e:
    print(f"Error: {e}")

# Multiple decorators on one function
from functools import wraps

def bold(func):
    @wraps(func)
    def wrapper(*args, **kwargs):
        result = func(*args, **kwargs)
        return f"**{result}**"
    return wrapper

def uppercase(func):
    @wraps(func)
    def wrapper(*args, **kwargs):
        result = func(*args, **kwargs)
        return result.upper()
    return wrapper

# Decorators are applied bottom-up
@bold
@uppercase
def greet(name: str) -> str:
    return f"Hello, {name}!"

print(greet("Alice"))
# Equivalent to: bold(uppercase(greet))("Alice")
# 1. greet("Alice") -> "Hello, Alice!"
# 2. uppercase applied -> "HELLO, ALICE!"
# 3. bold applied -> "**HELLO, ALICE!**"

# Order matters: @bold @uppercase vs @uppercase @bold
from functools import wraps

def bold(func):
    @wraps(func)
    def wrapper(*args, **kwargs):
        result = func(*args, **kwargs)
        return f"**{result}**"
    return wrapper

def uppercase(func):
    @wraps(func)
    def wrapper(*args, **kwargs):
        result = func(*args, **kwargs)
        return result.upper()
    return wrapper

@uppercase
@bold
def greet2(name: str) -> str:
    return f"Hello, {name}!"

print(greet2("Bob"))
# 1. greet2("Bob") -> "Hello, Bob!"
# 2. bold applied -> "**Hello, Bob!**"
# 3. uppercase applied -> "**HELLO, BOB!**"